# Explore here

In [70]:
!pip install missingno lazypredict
!pip install statsmodels

In [71]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import missingno as msno
from sklearn.impute import KNNImputer
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import xgboost as xgb
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import LabelEncoder

In [72]:
#1) Descargo los datos de bank:

#a) Leo los datos de la URL.
url = "https://raw.githubusercontent.com/4GeeksAcademy/linear-regression-project-tutorial/main/medical_insurance_cost.csv"

#b) Guardo los datos en un DataFrame.
df = pd.read_csv(url, sep=',')

#c) Miro las primeras filas del DF.
print(df.head())

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520


In [73]:
# Guardo el DataFrame.
df.to_csv('/workspace/Proyecto15_albaebauluz/data/raw/medical.csv', index=False)
#Configuro la biblioteca de Pandas en Python para controlar cómo se muestra el DF cuando los visualizo en un entorno interactivo (Jupyter Notebook). Nota: "None" sirve para que pandas muestre el total de las columnas aunque sean muchas. 
pd.set_option('display.max_columns', None)

df.head()
df.tail()

,age,sex,bmi,children,smoker,region,charges
1333,50,male,30.97,3,no,northwest,10600.5483
1334,18,female,31.92,0,no,northeast,2205.9808
1335,18,female,36.85,0,no,southeast,1629.8335
1336,21,female,25.80,0,no,southwest,2007.9450
1337,61,female,29.07,0,yes,northwest,29141.3603


In [74]:
# Tamaño de la muestra.
df.shape   

# Información de los datos.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [75]:
# Compruebo se hay valores duplicados. 
df.duplicated().sum()

np.int64(1)

In [76]:
# Muestro los id de los valores duplicados.
duplicados=np.where(df.duplicated())
print(duplicados)

(array([581]),)


In [77]:
# Visualizo la información de los id duplicados.
df.iloc[duplicados]

,age,sex,bmi,children,smoker,region,charges
581,19,male,30.59,0,no,northwest,1639.5631


In [78]:
# Elimino los 12 id duplicados. Nota: "inplace=True" indica que los elimine de la BD original.
df.drop_duplicates(inplace=True)

In [79]:
# Compruebo se hay valores duplicados. 
df.duplicated().sum()

np.int64(0)

In [80]:
# Análisis descriptivo de la muestra.
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1337.0,39.222139,14.044333,18.0000,27.000,39.0000,51.00000,64.00000
bmi,1337.0,30.663452,6.100468,15.9600,26.290,30.4000,34.70000,53.13000
children,1337.0,1.095737,1.205571,0.0000,0.000,1.0000,2.00000,5.00000
charges,1337.0,13279.121487,12110.359656,1121.8739,4746.344,9386.1613,16657.71745,63770.42801


In [81]:

#1) Creo una instancia de LabelEncoder.
le= LabelEncoder()

#2) Codifico las columnas categóricas no binarias.
#df["region"]= le.fit_transform(df["region"])
df["sex"]= le.fit_transform(df["sex"])
df["smoker"]= le.fit_transform(df["smoker"])

df.head()


,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [82]:
# Crear dummy para las cuatro regiones.
#noreste, sureste, suroeste, noroeste

# Crear variables dummy para cada categoría en 'region'
df_dummies = pd.get_dummies(df['region'], prefix='region')

# Unir las nuevas columnas dummy al DataFrame original
df = pd.concat([df, df_dummies], axis=1)

# Visualizar el DataFrame con las nuevas variables dummy
print(df.head())

   age  sex     bmi  children  smoker     region      charges  \
0   19    0  27.900         0       1  southwest  16884.92400   
1   18    1  33.770         1       0  southeast   1725.55230   
2   28    1  33.000         3       0  southeast   4449.46200   
3   33    1  22.705         0       0  northwest  21984.47061   
4   32    1  28.880         0       0  northwest   3866.85520   

   region_northeast  region_northwest  region_southeast  region_southwest  
0             False             False             False              True  
1             False             False              True             False  
2             False             False              True             False  
3             False              True             False             False  
4             False              True             False             False  


In [84]:
df['region_northeast']=df['region_northeast'].astype(int)
df['region_northwest']=df['region_northwest'].astype(int)
df['region_southeast']=df['region_southeast'].astype(int)
df['region_southwest']=df['region_southwest'].astype(int)

print(df.head())

   age  sex     bmi  children  smoker     region      charges  \
0   19    0  27.900         0       1  southwest  16884.92400   
1   18    1  33.770         1       0  southeast   1725.55230   
2   28    1  33.000         3       0  southeast   4449.46200   
3   33    1  22.705         0       0  northwest  21984.47061   
4   32    1  28.880         0       0  northwest   3866.85520   

   region_northeast  region_northwest  region_southeast  region_southwest  
0                 0                 0                 0                 1  
1                 0                 0                 1                 0  
2                 0                 0                 1                 0  
3                 0                 1                 0                 0  
4                 0                 1                 0                 0  


In [85]:
X = df.drop(['charges', 'region', 'region_northeast'], axis=1)
y = df['charges']
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.0
Date:                Wed, 21 Aug 2024   Prob (F-statistic):               0.00
Time:                        19:19:19   Log-Likelihood:                -13538.
No. Observations:                1337   AIC:                         2.709e+04
Df Residuals:                    1328   BIC:                         2.714e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.194e+04    988.227  

En primer lugar, modelo especificado explica el 75% de la variabilidad del coste del seguro médico.
Los coeficientes de las variables explicativas indican que todas son significativas excepto el sexto.
Si la edad se incrementa en un año, el coste del seguro médico se incrementa, en promedio, en 257.2$.
Si el índice de masa corporal (bmi) se incrementa en una unidad, el coste del seguro médico se incrementa, en promedio, en 332.6$.
Tener un hijo adicional incrementa el coste del seguro médico se incrementa, en promedio, en 478.8$.
Fumar incrementa el coste del seguro médico, en promedio, en 23820$.
Si la edad se incrementa en un año, el coste del seguro médico se incrementa, en promedio, en 257.2$.
Una persona que vivie en la región northwest tiene un coste del seguro médico de 349.2$ menos que una persona que vive en la región region_northeast (categoría de referencia).
Una persona que vivie en la región southeast tiene un coste del seguro médico de 1035.3$ menos que una persona que vive en la región region_northeast (categoría de referencia).
Una persona que vivie en la región southwest tiene un coste del seguro médico de 960.1$ menos que una persona que vive en la región region_northeast (categoría de referencia).
